In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import tensorflow as tf
tf.reset_default_graph()

def dynamic_rnn_test():

    vocab_size = 5
    SOS_token = 0
    EOS_token = 4
    
    x_data = np.array([[SOS_token, 3, 1, 2, 3, 2],[SOS_token, 3, 1, 2, 3, 1],[SOS_token, 1, 3, 2, 2, 1]], dtype=np.int32)
    y_data = np.array([[1,2,0,3,2,EOS_token],[3,2,3,3,1,EOS_token],[3,1,1,2,0,EOS_token]],dtype=np.int32)
    Y = tf.convert_to_tensor(y_data)
    print("data shape: ", x_data.shape)
    sess = tf.InteractiveSession()
    
    output_dim = vocab_size
    batch_size = len(x_data)
    hidden_dim =6
    num_layers = 2
    seq_length = x_data.shape[1]
    embedding_dim = 8

    init = np.arange(vocab_size*embedding_dim).reshape(vocab_size,-1)
    
    with tf.variable_scope('test') as scope:
        cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_dim)
        cell = tf.contrib.rnn.OutputProjectionWrapper(cell,output_dim)
    
        embedding = tf.get_variable("embedding", initializer=init.astype(np.float32),dtype = tf.float32)
        inputs = tf.nn.embedding_lookup(embedding, x_data) # batch_size  x seq_length x embedding_dim
    
        initial_state = cell.zero_state(batch_size, tf.float32) #(batch_size x hidden_dim) 
        outputs, last_state = tf.nn.dynamic_rnn(cell,inputs,sequence_length=[seq_length]*batch_size,initial_state=initial_state)    

        weights = tf.ones(shape=[batch_size,seq_length])
        loss =   tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
    
        sess.run(tf.global_variables_initializer())
        print("initial_state: ", sess.run(initial_state))
        print("\n\noutputs: ",outputs)
        o = sess.run(outputs)  #batch_size, seq_length, outputs
        o2 = sess.run(tf.argmax(outputs,axis=-1))
        print("\n",o,o2) #batch_size, seq_length, outputs
    
        print("\n\nlast_state: ",last_state)
        print(sess.run(last_state)) # batch_size, hidden_dim
      
        p = sess.run(tf.nn.softmax(outputs)).reshape(-1,output_dim)
        print("loss: {:20.6f}".format(sess.run(loss)))
        print("manual cal. loss: {:0.6f} ".format(np.average(-np.log(p[np.arange(y_data.size),y_data.flatten()]))) )

if __name__ == '__main__':
    dynamic_rnn_test()
    print('Done')

data shape:  (3, 6)
initial_state:  [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


outputs:  Tensor("test/rnn/transpose_1:0", shape=(3, 6, 5), dtype=float32)

 [[[-0.12756711 -1.8893006  -0.25830954 -0.45842457  0.45872414]
  [-0.69116426 -0.46647018 -1.6921908  -0.7897278  -0.6824441 ]
  [-0.6553759  -0.42732227 -1.7135938  -0.8279202  -0.649617  ]
  [-0.89303714 -0.7180878  -1.5440084  -0.56051016 -0.85721946]
  [-1.0946566  -0.9693808  -1.3960197  -0.33158404 -1.0317781 ]
  [-0.9855349  -0.8333744  -1.4761151  -0.45548484 -0.9373025 ]]

 [[-0.12756711 -1.8893006  -0.25830954 -0.45842457  0.45872414]
  [-0.69116426 -0.46647018 -1.6921908  -0.7897278  -0.6824441 ]
  [-0.6553759  -0.42732227 -1.7135938  -0.8279202  -0.649617  ]
  [-0.89303714 -0.7180878  -1.5440084  -0.56051016 -0.85721946]
  [-1.0946566  -0.9693808  -1.3960197  -0.33158404 -1.0317781 ]
  [-0.76414216 -0.56043863 -1.63591    -0.7054921  -0.74457157]]

 [[-0.12756711 -1.8893006  -0.25830954 -0.4584245